# 卷积网络

## 网络的训练过程

- 定义一些可学习参数（或权重）的神经网络
- 遍历输入数据
- 将输入数据通过神经网络处理获得输出
- 计算损失（距离正确结果有多远）
- 将梯度传播回网络
- 使用一定的规则更新参数，通常使用简单规则 `weight = weight - learning_rate * gradient`

## 定义网络

下面定义这个网络：

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # input, inchannel = 1, outchannel = 6, size = 5*5
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # operation
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


模型的可学习参数

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


尝试一个 32*32 的输入

In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)


tensor([[-0.0928,  0.0560, -0.0734, -0.0205,  0.0134, -0.0919, -0.0039, -0.0048,
          0.1046,  0.1780]], grad_fn=<AddmmBackward>)


In [4]:
# net.zero_grad()
# out.backward(torch.randn(1, 10))
output = net(input)

target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)

print(loss)

tensor(1.6087, grad_fn=<MseLossBackward>)


反向传播的调用链

In [5]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## 反向传播

需要先清除现有的梯度，否则梯度将会积累

In [6]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0152,  0.0493,  0.0057, -0.0079,  0.0013, -0.0108])


## 更新权重

这里使用梯度下降(SGD):

> weight = weight - learning_rate * gradient

In [7]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([-0.0152,  0.0493,  0.0057, -0.0079,  0.0013, -0.0108])
conv1.bias.grad after backward
tensor([-0.0152,  0.0493,  0.0057, -0.0079,  0.0013, -0.0108])
